In [10]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hyperparameter optimization") \
  .getOrCreate()

In [11]:
data = spark.read.csv("data/hotel-reservations.csv", header=True, inferSchema=True)

In [12]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  29040
Test size:  7235


In [13]:
stages = []

In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder 

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')
    
    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec')
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()
    
    stages += [string_indexer, encoder]

StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [15]:
stages += [StringIndexer(inputCol='booking_status', outputCol= 'booking_status_index')]

In [16]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [17]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [18]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [19]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

stages.append(assembler)

In [20]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol='features', labelCol='booking_status_index', maxIter=10)

stages.append(gbt)

In [21]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=stages)

In [22]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.stepSize, [0.1, 0.2]) \
    .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol='booking_status_index',
    predictionCol='prediction',
    metricName='accuracy'
)

tvs_model = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    seed=42
)

best_model = tvs_model.fit(train_data)

24/10/14 22:57:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/14 22:57:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1009.7 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1043.9 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1038.7 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1039.2 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1039.7 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1041.0 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1043.2 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary with size 1046.9 KiB
24/10/14 22:58:21 WARN DAGScheduler: Broadcasting large task binary wit

In [23]:
predictions = best_model.transform(test_data)

In [24]:
accuracy = evaluator.evaluate(predictions) * 100
print(f'Accuracy = {accuracy:.2f}%')

Accuracy = 87.84%


24/10/14 22:58:45 WARN DAGScheduler: Broadcasting large task binary with size 1951.3 KiB


In [25]:
for param_values, metric_value in zip(paramGrid, best_model.validationMetrics):
    for param, value in param_values.items():
        print(f'{param.name} = {value}')
    print(f'Accuracy -> {metric_value}')


maxDepth = 5
maxIter = 10
stepSize = 0.1
Accuracy -> 0.8458034235229155
maxDepth = 5
maxIter = 10
stepSize = 0.2
Accuracy -> 0.8522915516289343
maxDepth = 5
maxIter = 20
stepSize = 0.1
Accuracy -> 0.8521535063500828
maxDepth = 5
maxIter = 20
stepSize = 0.2
Accuracy -> 0.8611264494754279
maxDepth = 10
maxIter = 10
stepSize = 0.1
Accuracy -> 0.8728602981778023
maxDepth = 10
maxIter = 10
stepSize = 0.2
Accuracy -> 0.871893981225842
maxDepth = 10
maxIter = 20
stepSize = 0.1
Accuracy -> 0.8742407509663169
maxDepth = 10
maxIter = 20
stepSize = 0.2
Accuracy -> 0.8738266151297626
